In [1]:
import os
import cv2
import random
# import pathlib
import pathlib
import numpy as np
import multiprocessing
from sklearn import svm


from matplotlib import pyplot as plt
from skimage.morphology import skeletonize
from skimage.feature import hog
from tqdm.notebook import tqdm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score



In [2]:
lp_file_path = "./files/LP_COORDS.npz"

In [ ]:
cpu_count = multiprocessing.cpu_count()

In [3]:
def get_lp_coord(image_name):
    lp_coords = np.load(lp_file_path, allow_pickle = True)
    return lp_coords['coords'].tolist()[image_name]

In [4]:
def get_all_images(input_dir, ext = "jpg", shuffle = True):
    image_paths = [i for i in pathlib.Path(input_dir).rglob(".".join(["*", ext]))]
    if shuffle:
        random.shuffle(image_paths)
    return image_paths

In [5]:
def mean_smooth(input_array, win_size = 10):
    step = win_size //2 
    len_array = len(input_array)
    return_array = []
    for index, value in enumerate(input_array):
        i_range = [ max(0, index - step), min(len_array, index + step)]
        return_array.append(np.mean(input_array[i_range[0] : i_range[1] + 1]))
    return np.asarray(return_array)

In [40]:
def list_split(input_list, ratio = [0.5, 0.5], shuffle = False):
    assert sum(ratio) == 1, print("sum of ratio should be 1")
    input_list = sorted(input_list)
    len_list = len(input_list)
    
    split_len = []
    for i in ratio[:-1]:
        split_len.append(round(len_list*i))
    
    return_list = []
    temp = 0
    for i in split_len:
        start, end = temp, temp + i
        temp += i
        return_list.append(input_list[start: end])
    return_list.append(input_list[temp:])
        
    return return_list
    
    

In [6]:
def imread(file_name, is_gray = False, resize = False, width = None, height = None):
    if is_gray:
        image = cv2.imread(str(file_name), flags = cv2.IMREAD_GRAYSCALE)
    else:
        
        image = cv2.cvtColor(cv2.imread(str(file_name)), cv2.COLOR_BGR2RGB)
    
    if resize:
        h, w = image.shape[:2]
        if width == None and height != None:
            width = int(height/h*w)
        elif width != None and height == None:
            height = int(width/w*width)

        image = cv2.resize(image, (width, height))
        
    return image


def resize(input_image, is_gray = True, width = None, height = None):
    if is_gray:
        image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    h, w = image.shape[:2]
    if width == None and height != None:
        width = int(height/h*w)
    elif width != None and height == None:
        height = int(width/w*width)

    image = cv2.resize(image, (width, height))
    
    return image


In [7]:
winSize = (320, 160)
blockSize = (40, 40)
blockStride = (20, 20)
cellSize = (20,20)
nbins = 9
derivAperture = 1
winSigma = -1.
histogramNormType = 1
L2HysThreshold = 0.2
gammaCorrection = 1
nlevels = 64
signedGradients = True

cvhog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,
                        derivAperture,winSigma,histogramNormType,L2HysThreshold,
                        gammaCorrection,nlevels,signedGradients)

In [8]:
def roi_extraction_1(input_image):
    image_name = input_image.name
    image = imread(str(input_image))
    h, w = image.shape[:2]
    x1, y1, x2, y2 = get_lp_coord(image_name)
    lp_w, lp_h = x2 - x1, y2 - y1
    
    roi_x1, roi_x2 = max(0, int(x1 - 0.8*lp_w)), min(int(x2 + 0.8*lp_w), w)
    roi_y1, roi_y2 = max(0, int(y1 - 0.32*(roi_x2 - roi_x1))), y1
    
    return [x1, y1, x2, y2], [roi_x1, roi_y1, roi_x2, roi_y2], image[roi_y1:roi_y2, roi_x1:roi_x2]
    

In [ ]:
def roi_extraction_2(input_image):
    image_name = input_image.name
    image = imread(str(input_image))
    h, w = image.shape[:2]
    x1, y1, x2, y2 = get_lp_coord(image_name)
    lp_w, lp_h = x2 - x1, y2 - y1
    
    roi_x1, roi_x2 = max(0, int(x1 - 0.84*lp_w)), min(int(x2 + 0.84*lp_w), w)
    roi_y1, roi_y2 = max(0, int(y1 - 3.67*lp_h)), min(int(y2 + 1.53*lp_h), h)
    
    return [x1, y1, x2, y2], [roi_x1, roi_y1, roi_x2, roi_y2], image[roi_y1:roi_y2, roi_x1:roi_x2]
    

In [ ]:
def overall_accuracy(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

def mean_accuracy(y_true, y_pred):
    if not isinstance(y_true, np.ndarray):
        y_true = np.asarray(y_true)
    if not isinstance(y_pred, np.ndarray):
        y_pred = np.asarray(y_pred)

    unique_label = np.unique(y_true)
    per_class_acc = []
    for label in unique_label:
        index = np.where(y_true == label)
        per_class_acc.append(np.mean(y_true[index] == y_pred[index]))
    return np.mean(per_class_acc, dtype=np.float32)


def mean_prec(y_true, y_pred):
    return precision_score(y_true, y_pred, average = "macro")

def overall_prec(y_true, y_pred):
    if not isinstance(y_true, np.ndarray):
        y_true = np.asarray(y_true)
    if not isinstance(y_pred, np.ndarray):
        y_pred = np.asarray(y_pred)

    return np.mean(y_true == y_pred, dtype=np.float32)


def mean_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average = "macro")